In [ ]:
import sys, site, platform
print("Python:", sys.version)
print("Executable:", sys.executable)
print("Site-packages:", site.getsitepackages() if hasattr(site, "getsitepackages") else "n/a")
print("Platform:", platform.platform(), platform.machine())
try:
    import torch
    print("torch:", torch.__version__, "from", torch.__file__)
except Exception as e:
    print("Import torch failed:", e)

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

In [ ]:
import os
import requests

if not os.path.exists("the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)

with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
len(raw_text)

In [ ]:
import re

text = "Hello, world. This, is a test."

# Spliting by whitespace

result = re.split(r'(\s)', text)

print(result)

In [ ]:
# Splitting by punctuation

result = re.split(r'([,.]|\s)', text)
print(result)

In [ ]:
# Stripping whitespace

result = [item for item in result if item.strip()]
print(result)

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

In [29]:
len(preprocessed)

4690

## 2.3 Converting tokens into token IDs

In [31]:
vocab = sorted(set(preprocessed))
vocab_size = len(vocab)

In [32]:
vocab = {token:integer for integer, token in enumerate(vocab)}

In [34]:
# Simple Tokenizer class to do the work of taking in a text, then replacing the words with their token ID representations

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        # Inversion of vocab mapping 
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [35]:
tokenizer = SimpleTokenizerV1(vocab)

In [36]:
text = """It's the last he painted, you know, Mrs. Gisburn said with pardonable pride."""

In [37]:
ids = tokenizer.encode(text)

In [38]:
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 67, 7, 38, 851, 1108, 754, 793, 7]


In [39]:
tokenizer.decode(ids)

"It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride."

## 2.4 Adding special context tokens